In [130]:
!pip install sklearn
import nltk
nltk.download('punkt')
!pip install googletrans
!pip install  GetOldTweets3
!pip install langdetect
!pip install nltk
!pip install NRCLex
!pip install Unidecode
import GetOldTweets3 as got
from langdetect import detect
from googletrans import Translator
import pandas as pd
import numpy as np
!pip install spacy
!pip install vaderSentiment

def get_tweets(hashtag):
    import GetOldTweets3 as got
    from nltk.tokenize import TweetTokenizer
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(hashtag) \
        .setSince("2020-01-01") \
        .setUntil("2020-05-01") \
        .setMaxTweets(200)
    # Creation of list that contains all tweets
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)
    # Creating list of chosen tweet data
    text_tweets = [[tweet.text] for tweet in tweets]
    text2 =[]
    txt=[]
    n=0
    #new list to store translated tweets
    length=len(text_tweets)
    for i in range(length):
    #some tweets are empty so try 
        try:
         lang = detect(text_tweets[i][0])
         if(lang!='en'):
           translator = Translator()
           translated = translator.translate(text_tweets[i][0])
           text2.append(translated.text) 
         else:
           text2.append(text_tweets[i][0])  
        except:
          n=i
    return text2

from nltk.tokenize import TweetTokenizer

text=[]
token=[]
token1=[]
text=get_tweets('IndiafightsCOVID19')
text1=get_tweets('lockdown')
for i in range(len(text)):
  ttk = TweetTokenizer()
  t=ttk.tokenize(text[i])
  token.append(t)
for i in range(len(text)):
  ttk = TweetTokenizer()
  t=ttk.tokenize(text[i])
  token1.append(t)



import re, string
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []
    tokens=[]
    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#…]|[!*\(\),]'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
        
    return cleaned_tokens
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
text=""
t=[]
at=[]
at1=[]
length=len(token)
for i in range(length):
    t=(remove_noise(token[i], stop_words))
    for i in range(len(t)): 
        at.append(t[i]) 
df=pd.DataFrame(at,columns=['Tweets'])
length1=len(token1)
for i in range(length1):
    t=(remove_noise(token[i], stop_words))
    for i in range(len(t)): 
        at.append(t[i]) 
df1=pd.DataFrame(at,columns=['Tweets'])

from nrclex import NRCLex
import random
data= []
i=0
src=[]
emo=[]
def nrc(df):
    for i in df.index: 
             text1=df['Tweets'][i]
             text_object = NRCLex(text1)
             data=(text_object.top_emotions)
             dt=random.choice(data)
             emo.append(dt[0])
    return emo
ret=nrc(df)
df2=pd.DataFrame(ret,columns=['emotions'])
df['emotions']=df2['emotions']
ret1=nrc(df1)
df2=pd.DataFrame(ret1,columns=['emotions1'])
df1['emotions']=df2['emotions1']

from nltk import FreqDist
freq_dist_pos = FreqDist(df['emotions'])
d=[]
d=freq_dist_pos.most_common(10)
#print(d)
df3=pd.DataFrame(d,columns=['emotion','number'])
df3 = df3.drop(df3[df3.emotion == 'positive'].index)
df3 = df3.drop(df3[df3.emotion == 'negative'].index)
#print(df3)
from project_lib import Project
project = Project(project_id='0f669ebe-6025-4e78-90c5-6c4f6af6a73b', project_access_token='p-2177ac7823dd38b3c5c0b5aea3410fbef3724245')
project.save_data("final_emotion_n.csv", df3.to_csv(index=False),overwrite=True)

from nltk import FreqDist
freq_dist_pos = FreqDist(df1['emotions'])
d=[]
d=freq_dist_pos.most_common(10)
#print(d)
df4=pd.DataFrame(d,columns=['emotion','number'])
df4 = df4.drop(df4[df4.emotion == 'positive'].index)
df4 = df4.drop(df4[df4.emotion == 'negative'].index)
from project_lib import Project
project = Project(project_id='0f669ebe-6025-4e78-90c5-6c4f6af6a73b', project_access_token='p-2177ac7823dd38b3c5c0b5aea3410fbef3724245')
project.save_data("lockdwn_emotion_n.csv", df3.to_csv(index=False),overwrite=True)

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
lst=[]
sent=[]
lst1=[]
sent1=[]
def sentiment_analyzer_scores(sentence):
        score = analyser.polarity_scores(sentence)
        
        if score['compound'] > 0 :
            t='positive'
            return t
        elif score['compound']== 0:
            t='neutral'
            return t
        else:
            t='negative'
            return t

for i in df.index:
    lst=sentiment_analyzer_scores(df['emotions'][i])
    sent.append(lst)
print(df['emotions'])
from nltk import FreqDist
freq_dist_pos = FreqDist(sent)
d=[]
d=freq_dist_pos.most_common(3)

df5=pd.DataFrame(d,columns=['sentiment','count'])
from project_lib import Project
project = Project(project_id='0f669ebe-6025-4e78-90c5-6c4f6af6a73b', project_access_token='p-2177ac7823dd38b3c5c0b5aea3410fbef3724245')
project.save_data("final_sentiment.csv", df5.to_csv(index=False),overwrite=True)

for i in df1.index:
    lst1=sentiment_analyzer_scores(df1['emotions'][i])
    sent1.append(lst1)
#print(sent1)
freq_dist_pos = FreqDist(sent1)
d1=[]
d1=freq_dist_pos.most_common(3)

df6=pd.DataFrame(d1,columns=['sentiment','count'])
#print(df6)
from project_lib import Project
project = Project(project_id='0f669ebe-6025-4e78-90c5-6c4f6af6a73b', project_access_token='p-2177ac7823dd38b3c5c0b5aea3410fbef3724245')
project.save_data("lockdown_sentiment.csv", df6.to_csv(index=False),overwrite=True)

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn import metrics
#creating labelEncoder
le = preprocessing.LabelEncoder()
# Converting string labels into numbers.
df['emotion_en']=le.fit_transform(df['emotions'])

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(df['Tweets'],df['emotion_en'], test_size=0.4)
vector = TfidfVectorizer(analyzer='word',ngram_range=(2,3), max_features=5000)
vector.fit(df['Tweets'])
Train_X=vector.transform(X_train)
Test_X=vector.transform(X_test)


from sklearn.neighbors import KNeighborsClassifier

#Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=10)

#Train the model using the training sets
knn.fit(Train_X, y_train)
#Predict the response for test dataset
y_pred = knn.predict(Test_X)


yp_test = le.inverse_transform(y_pred)
yp_train=le.inverse_transform(y_train)
f1=pd.DataFrame(yp_train)
f2=pd.DataFrame(yp_test)
frames = [f1,f2]
t1=[]
lt=[]
result = pd.concat(frames)
result.columns=['emo']
for i in result.index:
    t1=result['emo'][i]
    lt.append(t1)
from nltk import FreqDist
freq_dist_pos = FreqDist(result['emo'])
#print(freq_dist_pos
d=[]
d=freq_dist_pos.most_common(7)

[('negative', 1657), ('positive', 244), ('surprise', 216), ('trust', 211), ('fear', 209), ('anticip', 207), ('anger', 205)]
